In [1]:
import numpy as np
import pandas as pd
import argparse
import json
import pickle
import torch.distributed as dist

import os
import gc

import pysam

#
#install pytorch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 pytorch-cuda=11.7 torchtriton -c pytorch -c nvidia
#pip install triton==2.0.0.dev20221202 --force --no-dependencies
import torch
from torch.utils.data import DataLoader, IterableDataset
from transformers import PreTrainedTokenizerFast, DataCollatorForLanguageModeling
from DNABERT2.configuration_bert import BertConfig
from DNABERT2.bert_layers import BertForMaskedLM
import os

from tqdm import tqdm

import helpers.misc as misc                #miscellaneous functions
import helpers.train_eval as train_eval    #train and evaluation
 

%load_ext autoreload
%autoreload 2

In [48]:
def next_free_port( port=1024, max_port=65535 ):
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    while port <= max_port:
        try:
            sock.bind(('', port))
            sock.close()
            return port
        except OSError:
            port += 1
    raise IOError('no free ports')

next_free_port()

1024

In [2]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

In [3]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('\nCUDA device: GPU\n')
else:
    device = torch.device('cpu')
    print('\nCUDA device: CPU\n')
    #raise Exception('CUDA is not found')


CUDA device: GPU



In [4]:
datadir = '/lustre/groups/epigenereg01/workspace/projects/vale/mlm/'

In [5]:
class SeqDataset(IterableDataset):
    
    def __init__(self, fasta_fa, seq_df):
        
        if fasta_fa:
            self.fasta = pysam.FastaFile(fasta_fa)
        else:
             self.fasta = None

        self.seq_df = seq_df
        self.start = 0
        self.end = len(self.seq_df)
        
    def __len__(self):
        return len(self.seq_df)
                
    def __iter__(self):
        
        #worker_total_num = torch.utils.data.get_worker_info().num_workers
        #worker_id = torch.utils.data.get_worker_info().id
        
        for seq_idx in range(self.start,self.end):
            
            if self.fasta:
                seq = self.fasta.fetch(self.seq_df.iloc[seq_idx].seq_name).upper()
            else:
                seq = self.seq_df.iloc[seq_idx].seq.upper()
    
            #species_label = self.seq_df.iloc[idx].species_label
                    
            tokenized_seq = tokenizer(seq,
                                    add_special_tokens=True,
                                    truncation=True,
                                    return_special_tokens_mask=True,
                                    padding='max_length',
                                    max_length=input_params.max_tok_len,)
            
            tokenized_seq['targets'] = tokenized_seq['input_ids'].copy()
            yield tokenized_seq
                        
    def close(self):
        self.fasta.close()

def worker_init_fn(worker_id):
     worker_info = torch.utils.data.get_worker_info()
     dataset = worker_info.dataset  # the dataset copy in this worker process
     overall_start = dataset.start
     overall_end = dataset.end
     # configure the dataset to only process the split workload
     per_worker = int(np.ceil((overall_end - overall_start) / float(worker_info.num_workers)))
     worker_id = worker_info.id
     dataset.start = overall_start + worker_id * per_worker
     dataset.end = min(dataset.start + per_worker, overall_end)

In [6]:
input_params = misc.dotdict({})

input_params.max_tok_len = 256
input_params.val_fraction = 0.1

input_params.batch_size = 128
input_params.weight_decay = 1e-5
input_params.max_lr = 5e-4

input_params.fasta = datadir + 'dnabert2/fasta/chunk_1024_overlap_256.fa'

input_params.output_dir = './test/'
input_params.tot_epochs = 20
input_params.val_fraction = 0.1
input_params.validate_every = 1
input_params.save_at = [2]

In [7]:
world_size = 8#dist.get_world_size()
rank = 2#dist.get_rank()

assert input_params.batch_size % world_size == 0, 'batch size should be divisible by world size'

In [22]:
seq_df = pd.read_csv(input_params.fasta + '.fai', header=None, sep='\t', usecols=[0], names=['seq_name'])

#seq_df = seq_df.iloc[:3000]

seq_df['species_name'] = seq_df.seq_name.apply(lambda x:x.split(':')).apply(lambda x:x[-1] if len(x)==2 else x[1])

all_species = sorted(seq_df.species_name.unique())

if not input_params.species_agnostic:
    species_encoding = {species:idx for idx,species in enumerate(all_species)}
else:
    species_encoding = {species:0 for species in all_species}
    
seq_df['species_label'] = seq_df.species_name.map(species_encoding)

#seq_df = seq_df.sample(frac = 1., random_state = 1) #DO NOT SHUFFLE, otherwise too slow

In [23]:
tokenizer = PreTrainedTokenizerFast(tokenizer_file="DNABERT2/tokenizer.json",
mask_token = '[MASK]', pad_token = '[PAD]', sep_token = '[SEP]', cls_token = '[CLS]', unk_token = '[UNK]',)
tokenizer.deprecation_warnings["Asking-to-pad-a-fast-tokenizer"] = True

collate_fn = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

In [24]:
N_train = int(len(seq_df)*(1-input_params.val_fraction))       

num_samples_per_rank = N_train // world_size

train_idx = range(rank*num_samples_per_rank,(rank+1)*num_samples_per_rank)

train_df = seq_df.iloc[train_idx] 
test_df = seq_df.iloc[num_samples_per_rank*world_size:]

train_dataset = SeqDataset(input_params.fasta, train_df)
train_dataloader = DataLoader(dataset = train_dataset, batch_size = input_params.batch_size // world_size, 
                              num_workers = 1, worker_init_fn=worker_init_fn, collate_fn = collate_fn, shuffle = False)

test_dataset = SeqDataset(input_params.fasta, test_df)
test_dataloader = DataLoader(dataset = test_dataset, batch_size = input_params.batch_size // world_size, 
                             num_workers = 1,  worker_init_fn=worker_init_fn, collate_fn = collate_fn, shuffle = False)

In [11]:
gc.collect()
torch.cuda.empty_cache()

In [12]:
config = BertConfig.from_pretrained('DNABERT2/config.json')

model = BertForMaskedLM(config).to(device)

In [13]:
model_params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.Adam(model_params, betas=(0.9,0.98), eps=1e-6,
                             lr = input_params.max_lr, 
                             weight_decay = input_params.weight_decay)

In [14]:
last_epoch, last_iteration = 0, -1

if input_params.checkpoint:

    model = BertForMaskedLM.from_pretrained(input_params.checkpoint_dir)
    
    if os.path.isfile(input_params.checkpoint_dir + '/opimizer.pt'):
            optimizer.load_state_dict(torch.load(input_params.checkpoint_dir + '/opimizer.pt'))
        
    with open(config_save_base + "train_state.json", "w") as f:
        train_state = json.load(f) 
        last_iteration = train_state['last_iteration']
        epoch = train_state['last_epoch']

weights_dir = os.path.join(input_params.output_dir, 'checkpoints') #dir to save model weights at save_at epochs

if input_params.save_at:
    os.makedirs(weights_dir, exist_ok = True)

In [15]:
scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr = 0, max_lr = input_params.max_lr,
                                             step_size_up = 20000, step_size_down = 180000, cycle_momentum=False,
                                             last_epoch = last_iteration)

In [16]:
def metrics_to_str(metrics):
    loss, total_acc, masked_acc = metrics
    return f'loss: {loss:.4}, total acc: {total_acc:.3f}, masked acc: {masked_acc:.3f}'

In [17]:
#input_params.tot_epochs=5

In [21]:
from IPython.display import clear_output

clear_output()

#from utils.misc import print    #print function that displays time

if not input_params.test:

    for epoch in range(last_epoch+1, input_params.tot_epochs+1):

        print(f'EPOCH {epoch}: Training...')

        print(f'using train samples: {list(train_dataset.seq_df.index[[0,-1]])}')

        train_metrics = train_eval.model_train(model, optimizer, train_dataloader, device, scheduler=scheduler,
                            silent = False)

        last_iteration+=int(np.ceil(len(train_dataset)/train_dataloader.batch_size))
        
        print(f'epoch {epoch} - train ({last_iteration+1} iterations), {metrics_to_str(train_metrics)}')

        if epoch in input_params.save_at: #save model weights

            misc.save_model_weights(model, optimizer, weights_dir, epoch, last_iteration)

        if input_params.val_fraction>0 and ( epoch==input_params.tot_epochs or
                            (input_params.validate_every and epoch%input_params.validate_every==0)):

            print(f'EPOCH {epoch}: Validating...')

            val_metrics =  train_eval.model_eval(model, optimizer, test_dataloader, device,
                    silent = False)

            print(f'epoch {epoch} - validation, {metrics_to_str(val_metrics)}')
            

EPOCH 1: Training...
using train samples: [674, 1010]


acc: 0.013, masked acc: 0.023, loss: 7.806: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:03<00:00,  6.28it/s]


22
epoch 1 - train (286 iterations), loss: 7.806, total acc: 0.013, masked acc: 0.023
EPOCH 1: Validating...


acc: 0.0075, masked acc: 0.019, loss: 7.769: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:01<00:00, 13.31it/s]


epoch 1 - validation, loss: 7.769, total acc: 0.008, masked acc: 0.019
EPOCH 2: Training...
using train samples: [674, 1010]


...

epoch 4 - validation, loss: 7.644, total acc: 0.008, masked acc: 0.021
EPOCH 5: Training...
using train samples: [674, 1010]


acc: 0.013, masked acc: 0.025, loss: 7.649: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [00:03<00:00,  6.44it/s]


22
epoch 5 - train (374 iterations), loss: 7.649, total acc: 0.013, masked acc: 0.025
EPOCH 5: Validating...


acc: 0.0076, masked acc: 0.021, loss: 7.605: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 19/19 [00:01<00:00, 12.42it/s]

epoch 5 - validation, loss: 7.605, total acc: 0.008, masked acc: 0.021
